In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import numpy as np
from pandas import read_excel

In [2]:
df = pd.read_csv('./Decomp_1.tsv', sep="\t")


In [3]:
df = df.drop(columns=['GOLDEN:result', 'HINT:text', 'HINT:default_language'])

In [4]:
#добавил данные с результатами
de = read_excel('roadSigns_1.xlsx')

In [5]:
df = pd.merge(df, de, on=['INPUT:image'], how='inner')
df

,INPUT:image,OUTPUT:result,ASSIGNMENT:worker_id,INPUT:label,INPUT:task-id
0,http://sdcimages.s3.yandex.net/nirvana/5868dfc...,pedestrian,040267dc3213d9bb136605f74e9fd81f,no_stop,131
1,http://sdcimages.s3.yandex.net/nirvana/5868dfc...,no_stop,419c035390638e54057fefdb02dd2814,no_stop,131
2,http://sdcimages.s3.yandex.net/nirvana/5868dfc...,no_stop,8179b955d7706457ac3698566f59d9cf,no_stop,131
3,http://sdcimages.s3.yandex.net/nirvana/84d7947...,pedestrian,040267dc3213d9bb136605f74e9fd81f,no_stop,137
4,http://sdcimages.s3.yandex.net/nirvana/84d7947...,no_stop,419c035390638e54057fefdb02dd2814,no_stop,137
...,...,...,...,...,...
673,http://sdcimages.s3.yandex.net/nirvana/a32a065...,no_stop,1442435d91d56394f486ded242b61332,no_stop,104
674,http://sdcimages.s3.yandex.net/nirvana/a32a065...,no_stop,9525a461cb742096fe0b0f2bd0c89cd9,no_stop,104
675,http://sdcimages.s3.yandex.net/nirvana/f1917f0...,bus_stop,8c54a484fc400b6ba9e7933791b9a4ed,bus_stop,57
676,http://sdcimages.s3.yandex.net/nirvana/f1917f0...,bus_stop,1442435d91d56394f486ded242b61332,bus_stop,57


In [6]:
# Считаем TP, TN, FP, FN (по всем толокерам)
(df['INPUT:label'] +'_'+ df['OUTPUT:result']).value_counts()

main_road_main_road      134
give_way_give_way        134
pedestrian_pedestrian    133
bus_stop_bus_stop        132
no_stop_no_stop          131
no_stop_pedestrian         3
bus_stop_pedestrian        3
main_road_pedestrian       2
pedestrian_bus_stop        2
main_road_give_way         2
give_way_pedestrian        1
no_stop_bus_stop           1
dtype: int64

In [7]:
#Считаем результат по большинству ответов толокеров по каждой задаче, записываем в словарь w. Задача 'INPUT:image' и 'OUTPUT:result'
w = {}
N = 0
for i in df['INPUT:image'].unique():
    N += 1
    #print(N, i)
    #display(df[df['INPUT:image']== i]['OUTPUT:result'].value_counts())
    w[i] = df[df['INPUT:image']== i]['OUTPUT:result'].value_counts().index[0]
    #if df[df['INPUT:image']== i]['OUTPUT:result'].value_counts()[0] != 3:
    #    break

In [8]:
# Создаем df2 без перекрытия 
df2 = df[['INPUT:image','INPUT:label']].drop_duplicates()

In [9]:
# в df2 создаем столбец с разметкой толокеров, по большинству из словаря w
df2['tolokers'] =df2['INPUT:image'].map(w)

In [10]:
# Считаем TP, TN, FP, FN (по всем уникальным задачам, используя большинство) 
(df2['INPUT:label'] +' - '+ df2['tolokers']).value_counts()

no_stop - no_stop          45
main_road - main_road      45
bus_stop - bus_stop        45
give_way - give_way        45
pedestrian - pedestrian    45
dtype: int64